In [166]:
# !pip install -U torch transformers accelerate sentence-transformers
# !pip install -U torch transformers accelerate sentence-transformers
# !pip uninstall spellchecker -y
# !pip install pyspellchecker
# !pip install unidecode
# !pip install scikit-learn  # For cosine_similarity and numpy (it's a dependency)
# !pip install sentence-transformers
# !pip install pandas
#!pip install ipywidgets

import re
import unidecode
from spellchecker import SpellChecker
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import datadetox as detox
import importlib
importlib.reload(detox)

<module 'datadetox' from 'c:\\Users\\USER\\Desktop\\GitHub\\Brasileirao-Analise\\3. Source Code\\Transformed\\datadetox.py'>

Leitura e padronização de colunas de chaves primarias para concatenação de dfs

In [167]:
pd.set_option('display.max_columns', None)
path = r'C:\Users\USER\Desktop\GitHub\Brasileirao-Analise\1. Data\Transformed'

#Games 
df_list = []
for i in range(1, 8):
    df = pd.read_excel(path + f'\\0{i}_games.xlsx')
    df_list.append(df)

#fitragem série B comjumto 7
df_list[6] = df_list[6][df_list[6]['Série']=='A']

# mapeamento_colunas chave primária
mapeamento_colunas = {  'data': 'Data','Date': 'Data','data_partida': 'Data',
                        'time_mandante': 'Mandante','mandante': 'Mandante','Home': 'Mandante',
                        'time_visitante': 'Visitante','visitante': 'Visitante','Away': 'Visitante'}
    

df_list = [df.rename(columns=mapeamento_colunas) for df in df_list]

df_list = detox.padronizar_datas(df_list)


DataFrame 0: Já é datetime64, mantido como está.
DataFrame 1: Convertendo padrão com '/'
DataFrame 2: Convertendo padrão com '/'
DataFrame 3: Convertendo padrão com '/'
DataFrame 4: Já é datetime64, mantido como está.
DataFrame 5: Já é datetime64, mantido como está.
DataFrame 6: Convertendo padrão ISO com '-'


Unir todos os DFs com agrupamento por colunas, Data, Mandante e Visitante

In [ ]:
#mapear cluber qeu o filtro BERT nao resolve (manualmente)
clubes_map_inicial = {  "America-RN":"América Natal","América-RN":"América Natal","América rn":"América Natal",
                        "Joinvile":"Joinville",
                        "Atlético pr":"Athlético-PR","Atlético-PR":"Athlético-PR", 
                        "Grêmio Prudente":"Barueri","Grêmio prudente":"Barueri","Gremio Prudente":"Barueri"}

for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_map_inicial)
    df['Visitante'] = df['Visitante'].replace(clubes_map_inicial)

# listar clubes Mandantes e visitantes, para agrupar ortograficamente
clubes_unicos = []
for df in df_list:
    unicos_por_bd = pd.unique(np.concatenate([df['Mandante'].values, df['Visitante'].values])).tolist()
    clubes_unicos = list(set(clubes_unicos) | set(unicos_por_bd))

clubes_value_dict = [   "América Mineiro", "América Natal", "Athletico Paranaense", "Atlético Goianiense", "Atlético Mineiro", "Avaí", "Bahia", 
                        "Barueri", "Botafogo", "Red Bull Bragantino", "Brasiliense", "Ceará", 
                        "Chapecoense", "Corinthians", "Coritiba", "Criciúma", "Cruzeiro", "CSA", "Cuiabá", "Figueirense", "Flamengo", "Fluminense", 
                        "Fortaleza","Guarani", "Goiás", "Grêmio", "Grêmio Prudente", "Internacional", "Ipatinga", "Joinville", "Juventude","Mirassol", "Náutico", 
                        "Palmeiras", "Paraná", "Paysandu", "Ponte Preta", "Portuguesa", "Santa Cruz", "Santo André", "Santos", "São Caetano", "São Paulo", 
                        "Sport Recife", "Vasco da Gama", "Vitória"]

modelo = SentenceTransformer('all-MiniLM-L6-v2')
clubes_corrigidos = detox.corrigir_times_semanticamente(clubes_unicos, clubes_value_dict,modelo)
clubes_dict = detox.gerar_dicionario_mapeamento(clubes_corrigidos, limite=0.0)

# corrigindo e agrupando clubes em todos os dataframes
for df in df_list:
    df['Mandante'] = df['Mandante'].replace(clubes_dict)
    df['Visitante'] = df['Visitante'].replace(clubes_dict)

# unindo todos os dfs com merge on=['Data','Mandante','Visitante']
df_merged = pd.DataFrame(columns=['Data','Mandante','Visitante'])
for i,df in enumerate(df_list):
    df_merged = pd.merge(df_merged, df, how='outer', on=['Data','Mandante','Visitante'],suffixes=('', f'_{i}'))


In [ ]:
def merge_rows(df, key_columns, verbose=False):
    """
    Mescla linhas de um DataFrame onde as colunas chave são iguais, 
    seguindo as regras:
    - Se ambos valores forem NaN, manter NaN
    - Se um valor for NaN e outro não, manter o valor não-NaN
    - Se ambos tiverem valores diferentes, não mesclar essas linhas
    
    Args:
        df: DataFrame original
        key_columns: Lista com os nomes das colunas chave para agrupamento
        verbose: Se True, mostra mensagens detalhadas durante o processo
        
    Returns:
        DataFrame com as linhas mescladas
    """
    # Cria uma cópia para não modificar o DataFrame original
    result_df = df.copy()
    
    # Identifica as colunas que não são chaves
    non_key_columns = [col for col in df.columns if col not in key_columns]
    
    # Agrupa por colunas chave
    grouped = result_df.groupby(key_columns)
    
    # Lista para armazenar as linhas mescladas
    merged_rows = []
    conflicts = []
    
    # Processa cada grupo
    for name, group in grouped:
        # Se o grupo tem apenas uma linha, adiciona diretamente
        if len(group) == 1:
            merged_rows.append(group.iloc[0])
            continue
        
        # Processa grupos com múltiplas linhas
        merged_row = {}
        
        # Adiciona as colunas chave
        for i, col in enumerate(key_columns):
            if isinstance(name, tuple):
                merged_row[col] = name[i]
            else:
                merged_row[col] = name
        
        # Verifica conflitos nas outras colunas
        has_conflict = False
        
        for col in non_key_columns:
            # Obtém todos os valores não-NaN da coluna no grupo
            values = group[col].dropna().unique()
            
            # Caso 1: Todos são NaN
            if len(values) == 0:
                merged_row[col] = np.nan
            
            # Caso 2: Apenas um valor único não-NaN
            elif len(values) == 1:
                merged_row[col] = values[0]
            
            # Caso 3: Múltiplos valores diferentes
            else:
                has_conflict = True
                if verbose:
                    print(f"Conflito encontrado no grupo {name} na coluna {col}:")
                    print(f"Valores: {values}")
                conflicts.append(name)
                break
        
        if not has_conflict:
            merged_rows.append(merged_row)
        else:
            # Se há conflito, mantém as linhas originais
            for _, row in group.iterrows():
                merged_rows.append(row.to_dict())
    
    # Cria um novo DataFrame com as linhas mescladas
    result = pd.DataFrame(merged_rows)
    
    # Reordena as colunas para manter a mesma ordem do DataFrame original
    result = result[df.columns]
    
    if verbose and conflicts:
        print(f"Total de grupos com conflitos: {len(set(conflicts))}")
    
    return result



In [ ]:
import datetime as dt
df = df_merged.copy()

# Criar edições dos campeonatos visto que 2020 foi ate 2021
mask = (df['Data'] > dt.datetime(2020, 8, 1)) & (df['Data'] < dt.datetime(2021, 2, 28))
df.loc[mask, 'edition'] = 2020
df.loc[~mask, 'edition'] = df.loc[~mask, 'Data'].dt.year

print(df['edition'].value_counts())

df = df[df['edition'] == 2024]
duplicados = df[df.duplicated(subset=['Mandante', 'Visitante'], keep=False)]
#duplicados.sort_values(by=['Mandante', 'Visitante'], ascending=[True, False])
#display(duplicados.sort_values(by='Mandante'))





edition
2006.0    751
2004.0    730
2003.0    661
2021.0    589
2005.0    585
2010.0    584
2022.0    575
2018.0    568
2019.0    563
2020.0    562
2017.0    551
2016.0    530
2023.0    522
2013.0    521
2015.0    506
2014.0    497
2012.0    478
2007.0    461
2024.0    423
2008.0    391
2009.0    388
2011.0    380
Name: count, dtype: int64


C:\Users\USER\AppData\Local\Temp\ipykernel_14896\2588190267.py:29: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  return grupo.ffill().bfill().iloc[0]
C:\Users\USER\AppData\Local\Temp\ipykernel_14896\2588190267.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grupos = df.groupby(chaves, as_index=False).apply(mescla_segura).dropna().reset_index(drop=True)


In [202]:
print(grupos.shape)

(0, 160)
